In [0]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
!pip install fire

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My\ Drive/"
base_dir = root_dir + 'fastai-v3/les8'

Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/fastai-v3/les8

/content/gdrive/My Drive/fastai-v3/les8


In [0]:
pwd

'/content/gdrive/My Drive/fastai-v3/les8'

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
#export
from exp.nb_11 import *

Serializing the model

In [0]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [0]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [0]:
len(il)

12954

In [0]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [0]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [0]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [0]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.093581,0.266581,2.040565,0.298000,00:41
1,1.905417,0.364702,1.912667,0.358000,00:41
2,1.788388,0.422836,1.811056,0.394000,00:41
3,1.691867,0.466597,1.778573,0.444000,00:41
4,1.622469,0.497993,1.716102,0.440000,00:41
5,1.587610,0.518709,1.779425,0.410000,00:41
6,1.535733,0.542476,1.652970,0.470000,00:41
7,1.485398,0.566806,1.617192,0.518000,00:41
8,1.441484,0.585515,1.520658,0.532000,00:41
9,1.395870,0.610647,2.078566,0.394000,00:41


In [0]:
st = learn.model.state_dict()

In [0]:
type(st)

collections.OrderedDict

In [0]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [0]:
st['10.bias']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [0]:
torch.save(st, mdl_path/'iw5')

Pet

In [0]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [0]:
pets.ls()

[PosixPath('/root/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations')]

In [0]:
pets_path = pets/'images'

In [0]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [0]:
il

ImageList (7390 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/newfoundland_169.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/miniature_pinscher_112.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/saint_bernard_57.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/British_Shorthair_168.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bombay_208.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/newfoundland_62.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/pug_195.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/english_cocker_spaniel_86.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Maine_Coon_106.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Siamese_93.jpg')...]
Path: /root/.fastai/data/oxford-iiit-pet/images

In [0]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [0]:
random.seed(42)

In [0]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [0]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/newfoundland_169.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/saint_bernard_57.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/British_Shorthair_168.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bombay_208.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/newfoundland_62.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/pug_195.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Maine_Coon_106.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/american_pit_bull_terrier_63.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/beagle_75.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_171.jpg')...]
Path: /root/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/miniature_pinscher_112.jpg'), PosixPath('/root/.fastai/data/oxf

In [0]:
n = il.items[0].name; n

'newfoundland_169.jpg'

In [0]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'newfoundland'

In [0]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [0]:
proc = CategoryProcessor()

In [0]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [0]:
', '.join(proc.vocab)

'newfoundland, saint_bernard, British_Shorthair, Bombay, pug, Maine_Coon, american_pit_bull_terrier, beagle, Egyptian_Mau, Siamese, Persian, leonberger, Abyssinian, Bengal, scottish_terrier, Ragdoll, japanese_chin, wheaten_terrier, staffordshire_bull_terrier, miniature_pinscher, english_setter, chihuahua, basset_hound, havanese, american_bulldog, samoyed, keeshond, boxer, english_cocker_spaniel, great_pyrenees, shiba_inu, Russian_Blue, german_shorthaired, yorkshire_terrier, Birman, Sphynx, pomeranian'

In [0]:
ll.valid.x.tfms = val_tfms

In [0]:
c_out = len(proc.vocab)

In [0]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [0]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.468664,0.088496,3.460747,0.099585,00:41
1,3.268115,0.140243,4.135175,0.114799,00:41
2,3.039031,0.202640,3.542480,0.116183,00:41
3,2.742356,0.283336,2.937413,0.273859,00:41
4,2.409667,0.403480,2.529680,0.354080,00:41


Custom head

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [0]:
st = torch.load(mdl_path/'iw5')

In [0]:
m = learn.model

In [0]:
m.load_state_dict(st)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [0]:
xb,yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
pred = m_cut(xb)

In [0]:
pred.shape

torch.Size([128, 512, 4, 4])

In [0]:
ni = pred.shape[1]

In [0]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [0]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))

In [0]:
learn.model = m_new

In [0]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.908016,0.274036,2.210784,0.446750,00:43
1,1.963488,0.549423,2.067179,0.515906,00:43
2,1.727471,0.624419,1.916107,0.557400,00:43
3,1.479843,0.725514,1.564007,0.692946,00:42
4,1.298832,0.801110,1.491068,0.712310,00:42


adapt_model and gradual unfreezing

In [0]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children())
               if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [0]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.781728,0.295935,2.395727,0.420470,00:38
1,2.379545,0.430178,2.333800,0.448133,00:38
2,2.019339,0.536523,2.103325,0.522822,00:38


In [0]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [0]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.843378,0.599520,1.983186,0.550484,00:42
1,1.740428,0.636418,1.928875,0.556017,00:41
2,1.640212,0.670766,1.810069,0.612725,00:41
3,1.465630,0.744263,1.658422,0.651452,00:41
4,1.275518,0.815659,1.556656,0.690180,00:42


Batch norm transfer

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)

def set_grad(m, b):
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [0]:
apply_mod(learn.model, partial(set_grad, b=False))

In [0]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.651133,0.328634,2.285615,0.437068,00:40
1,2.048548,0.520324,2.117810,0.500692,00:41
2,1.816630,0.604170,1.870621,0.578147,00:41


In [0]:
apply_mod(learn.model, partial(set_grad, b=True))

In [0]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.702141,0.652167,1.845060,0.598894,00:42
1,1.660370,0.667767,1.827351,0.578147,00:41
2,1.589956,0.687566,1.855688,0.597510,00:42
3,1.424469,0.747263,1.581046,0.692946,00:43
4,1.266691,0.821509,1.524528,0.694329,00:43


Pytorch already has an apply method we can use:

In [0]:
learn.model.apply(partial(set_grad, b=False));

Discriminative LR and param groups

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [0]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

In [0]:
a,b = bn_splitter(learn.model)

In [0]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [0]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [0]:
cb_types.after_backward

'after_backward'

In [0]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [0]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [0]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [0]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.498148,0.376181,2.286370,0.439834,00:42
1,2.201760,0.471276,2.231653,0.438451,00:42
2,1.925105,0.566372,1.923331,0.561549,00:43


In [0]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [0]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.772800,0.624569,2.000865,0.535270,00:42
1,1.797345,0.605070,1.926381,0.558783,00:43
2,1.667963,0.655017,1.796210,0.614108,00:43
3,1.519491,0.711564,1.711256,0.630705,00:43
4,1.405331,0.752062,1.655521,0.655602,00:43


Export

In [0]:
!python notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to exp/nb_11a.py
